In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# LIBRARIES

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.neighbors import LocalOutlierFactor
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.exceptions import ConvergenceWarning
import warnings

# DATA SET SETTINGS and SETTINGS

In [ ]:
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# READ DATA SET

In [ ]:
train_df = pd.read_csv("../input/titanic/train.csv")
test_df = pd.read_csv("../input/titanic/test.csv")
gender_sub = pd.read_csv("../input/titanic/gender_submission.csv")
combine = [train_df, test_df]

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
gender_sub.head()

# FUNCTIONS

In [ ]:
# Data set observe
def check_df(dataframe):
    print("##################### Shape #####################")
    print(dataframe.shape)
    print("##################### Types #####################")
    print(dataframe.dtypes)
    print("##################### Head #####################")
    print(dataframe.head(3))
    print("##################### Tail #####################")
    print(dataframe.tail(3))
    print("##################### NA #####################")
    print(dataframe.isnull().sum())
    print("##################### Quantiles #####################")
    print(dataframe.quantile([0, 0.05, 0.50, 0.95, 0.99, 1]).T)

In [ ]:
# Missing value capturing variables
def missing_values_table(dataframe, na_name=False):
    na_columns = [col for col in dataframe.columns if dataframe[col].isnull().sum() > 0]

    n_miss = dataframe[na_columns].isnull().sum().sort_values(ascending=False)

    ratio = (dataframe[na_columns].isnull().sum() / dataframe.shape[0] * 100).sort_values(ascending=False)

    missing_df = pd.concat([n_miss, np.round(ratio, 2)], axis=1, keys=['n_miss', 'ratio'])

    print(missing_df, end="\n")

    if na_name:
        return na_columns

In [ ]:
# Functions for detecting and processing outliers
def outlier_thresholds(dataframe, col_name):
    quartile1 = dataframe[col_name].quantile(0.05)
    quartile3 = dataframe[col_name].quantile(0.95)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit


def check_outlier(dataframe, col_name):
    low_limit, up_limit = outlier_thresholds(dataframe, col_name)
    if dataframe[(dataframe[col_name] > up_limit) | (dataframe[col_name] < low_limit)].any(axis=None):
        return True
    else:
        return False


def replace_with_thresholds(dataframe, col_name):
    low_limit, up_limit = outlier_thresholds(dataframe, col_name)
    if low_limit > 0:
        dataframe.loc[(dataframe[col_name] < low_limit), col_name] = low_limit
        dataframe.loc[(dataframe[col_name] > up_limit), col_name] = up_limit
    else:
        dataframe.loc[(dataframe[col_name] > up_limit), col_name] = up_limit

In [ ]:
# Label Encoding and One-Hot Encoding functions for transformation
def label_encoder(dataframe, binary_col):
    labelencoder = preprocessing.LabelEncoder()
    dataframe[binary_col] = labelencoder.fit_transform(dataframe[binary_col])
    return dataframe


def one_hot_encoder(dataframe, categorical_cols, drop_first=False):
    dataframe = pd.get_dummies(dataframe, columns=categorical_cols, drop_first=drop_first)
    return dataframe

In [ ]:
# Missing vs Target 
def missing_vs_target(dataframe, target, na_columns):
    temp_df = dataframe.copy()
    for col in na_columns:
        temp_df[col + '_NA_FLAG'] = np.where(temp_df[col].isnull(), 1, 0)
    na_flags = temp_df.loc[:, temp_df.columns.str.contains("_NA_")].columns
    for col in na_flags:
        print(pd.DataFrame({"TARGET_MEAN": temp_df.groupby(col)[target].mean(),
                            "Count": temp_df.groupby(col)[target].count()}), end="\n\n\n")

# EDA

In [ ]:
# Train and test observation
for dataset in combine:
    print(check_df(dataset))
    print("-" * 30)


In [ ]:
# Train and test descriptive statistics
for dataset in combine:
    print(dataset.quantile([0.0, 0.01, 0.05, 0.5, 0.95, 0.99, 1]).T)
    print("-" * 30)

In [ ]:
# Train set numeric density
columns = [col for col in train_df.columns if train_df[col].dtypes != "O" and train_df[col].nunique() > 10]
for col in columns:
    sns.histplot(x=dataset[col])
    plt.title("DENSITY")
    plt.xlabel(col)
    plt.show()

# MISSING VALUES

In [ ]:
# Are there any missing observations?
for i in combine:
    print(i.isnull().values.any())
    print("-" * 20)

In [ ]:
# Number of missing observations in each variable
for i in combine:
    print(i.isnull().sum())
    print("-" * 20)


In [ ]:
# How many deficiencies are observed in the variables with missing observation values and the ratio within itself
for dataset in combine:
    print(dataset.shape)
    missing_values_table(dataset, na_name=True)
    print("-" * 30)


In [ ]:
# Let me investigate whether the missing values of the variables containing missing observation
# values are important according to the dependent variable.
# Since there is no TARGET variable in the test_df data set, it will be taken from the train_df data set.
na_cols = missing_values_table(train_df, na_name=True)
missing_vs_target(train_df, "Survived", na_cols)


Missing values in the CABIN variable have less chance of being salvaged on average.

# FILL IN MISSING OBSERVATIONS

In [ ]:
for dataset in combine:
    dataset["Age"].fillna(dataset.groupby("Sex")["Age"].transform("median"), inplace=True)

train_df["Embarked"].fillna(train_df["Embarked"].dropna().mode()[0], inplace=True)
test_df["Fare"].fillna(test_df["Fare"].dropna().median(), inplace=True)


In [ ]:
# A new variable has been created over the cabin variable.
for dataset in combine:
    dataset["NEW_CABIN_BOOL"] = dataset["Cabin"].isnull().astype('int')

In [ ]:
# Train and test set Cabin variable is deleted
for dataset in combine:
    dataset.drop("Cabin", axis=1, inplace=True)


In [ ]:
# Control
for dataset in combine:
    print(dataset.shape)
    print(dataset.isnull().values.any())


# Outlier Values

In [ ]:
# will be checked for the train set
num_cols = [i for i in train_df.columns if train_df[i].dtypes != "O" and train_df[i].nunique() > 10]

for i in num_cols:
    print(i, check_outlier(train_df, i))

replace_with_thresholds(train_df, "Fare")


for i in num_cols:
    print(i, check_outlier(train_df, i))

# Local Outlier Factor

In [ ]:
dff = train_df.select_dtypes(include=['float64', 'int64'])
clf = LocalOutlierFactor(n_neighbors=20)
clf.fit_predict(dff)
df_scores = clf.negative_outlier_factor_

scores = pd.DataFrame(np.sort(df_scores))
scores.plot(stacked=True, xlim=[0, 20], style='.-')
plt.show()

In [ ]:
# remove outlier 
esik_deger = np.sort(df_scores)[5]
train_df.drop(labels=train_df[df_scores < esik_deger].index, axis=0, inplace=True)

# CATEGORICAL VS TARGET ANALYSIS

In [ ]:

train_cat_cols = [i for i in train_df.columns if train_df[i].nunique() < 5 and i not in "Survived"]

for i in train_cat_cols:
    print(train_df.groupby(i).agg({"Survived": ["mean", "count"]}))
    print("-" * 20)


# NUMERICAL VS TARGET ANALYSIS

In [ ]:

for i in num_cols:
    print(train_df.groupby("Survived").agg({i: ["mean"]}))


# FEATURE ENGINEERING

In [ ]:

for dataset in combine:
    dataset.loc[((dataset['SibSp'] + dataset['Parch']) > 0), "NEW_IS_ALONE"] = "NO"
    dataset.loc[((dataset['SibSp'] + dataset['Parch']) == 0), "NEW_IS_ALONE"] = "YES"

for dataset in combine:
    dataset["NEW_NAME_COUNT"] = dataset["Name"].str.len()
    dataset["NEW_NAME_WORD_COUNT"] = dataset["Name"].apply(lambda x: len(str(x).split(" ")))
    dataset["NEW_FAMILY_SIZE"] = dataset["SibSp"] + dataset["Parch"] + 1

for dataset in combine:
    dataset["NEW_AGExPCLASS"] = dataset["Age"] * dataset["Pclass"]

for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)

for dataset in combine:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess', 'Capt', 'Col', \
                                                 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)

# COLUMNS UPPER
for dataset in combine:
    dataset.columns = [col.upper() for col in dataset.columns]


# Label Encoding

In [ ]:

for dataset in combine:
    binary_cols = [col for col in dataset.columns if len(dataset[col].unique()) == 2 and dataset[col].dtypes == 'O']
    for i in binary_cols:
        label_encoder(dataset, i)

print(f"Train: {train_df.shape} \nTest: {test_df.shape}")


# One-Hot Encoding

In [ ]:

ohe_cols = [col for col in train_df.columns if 5 >= len(train_df[col].unique()) > 2]
train_df = one_hot_encoder(train_df, ohe_cols)

ohe_cols = [col for col in test_df.columns if 5 >= len(test_df[col].unique()) > 2]
test_df = one_hot_encoder(test_df, ohe_cols)

print(f"Train: {train_df.shape} \nTest: {test_df.shape}")


# SET VARIABLE

In [ ]:
train_df.drop(['TICKET', 'NAME', "PASSENGERID"], axis=1, inplace=True)
X = train_df.drop("SURVIVED", axis=1)
y = train_df["SURVIVED"]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=42)


In [ ]:
X_test = test_df.drop(['PASSENGERID', 'TICKET', 'NAME'], axis=1).copy()
y_test = gender_sub["Survived"]


# MODELS

In [ ]:
models = {"RANDOM FORREST CLASS": RandomForestClassifier(),
          "LGBM CLASS": LGBMClassifier(),
          "GRADIENT BOOSTING CLASS": GradientBoostingClassifier()}

for model in models.keys():
    fit = models[model].fit(X_train, y_train)
    print(f"{model} : {accuracy_score(y_val, fit.predict(X_val))}")


In [ ]:
for model in models.keys():
    fit = models[model].fit(X_train, y_train)
    print(f"{model} : {accuracy_score(y_test, fit.predict(X_test))}")

# RANDOM FORREST TUNING

In [ ]:
rf_params = {"max_depth": [3, 5, 8, None],
             "max_features": [3, 5, 15, 20],
             "n_estimators": [1000, 2000],
             "min_samples_split": [2, 5, 8]}

rfm_model = RandomForestClassifier(random_state=123)
rfm_cv_model = GridSearchCV(rfm_model, rf_params, cv=5, n_jobs=-1, verbose=1).fit(X_train, y_train)

rf_tuned = RandomForestClassifier(**rfm_cv_model.best_params_).fit(X_train, y_train)
y_pred = rf_tuned.predict(X_val)
accuracy_score(y_val, y_pred)
accuracy_score(y_test, rf_tuned.predict(X_test))

# LGBM CLASS TUNING

In [ ]:
lgb_model = LGBMClassifier()

lgbm_params = {"learning_rate": [0.01, 0.1],
               "n_estimators": [500, 1000],
               "max_depth": [3, 5, 8],
               "colsample_bytree": [1, 0.8, 0.5]}

lgbm_cv_model = GridSearchCV(lgb_model,
                             lgbm_params,
                             cv=10,
                             n_jobs=-1,
                             verbose=2).fit(X_train, y_train)


lgbm_tuned = LGBMClassifier(**lgbm_cv_model.best_params_).fit(X_train, y_train)
y_pred = lgbm_tuned.predict(X_val)
np.sqrt(accuracy_score(y_val, y_pred))
accuracy_score(y_test, lgbm_tuned.predict(X_test))

# GRADIENT BOOSTING CLASS TUNING

In [ ]:
gbm_params = {"learning_rate": [0.01, 0.1],
              "max_depth": [3, 8],
              "n_estimators": [500, 1000],
              "subsample": [1, 0.5, 0.7]}

gbm_model = GradientBoostingClassifier(random_state=17)
gbm_cv_model = GridSearchCV(gbm_model, gbm_params, cv=10, n_jobs=-1, verbose=2).fit(X_train, y_train)

gbm_tuned = GradientBoostingClassifier(**gbm_cv_model.best_params_).fit(X_train, y_train)
y_pred = gbm_tuned.predict(X_val)
np.sqrt(accuracy_score(y_val, y_pred))
accuracy_score(y_test, gbm_tuned.predict(X_test))
